# Broadcasting in `numpy` and `pytorch`

Example of using broadcasting to apply the same convolution filter (eg. guassian filter of size 3x3) to all channels


## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys
import re

sys.dont_write_bytecode = True

In [ ]:
import pandas as pd
# import geopandas as gpd
import joblib

import numpy as np
import matplotlib.pyplot as plt
import cv2

from pprint import pprint
from pathlib import Path

import pdb

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
    opts.Curve( tools=['hover'], active_tools=['wheel_zoom'], padding=0.1),

)

In [ ]:
this_nb_path = Path(os.getcwd())
ROOT = this_nb_path.parent
SCRIPTS = ROOT/'codes'
paths2add = [this_nb_path, SCRIPTS]

print("Project root: ", str(ROOT))
print("this nb path: ", str(this_nb_path))
print('Scripts folder: ', str(SCRIPTS))

for p in paths2add:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(str(p), "added to the path\n")
        
# print(sys.path)

In [ ]:
import ipywidgets
from ipywidgets import interact
def f(x):
    return x

interact(f, x=10)

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

## Tile n-dim array to (n+1) dim
Use `np.broadcast_to(arr, (n_times, *arr.shape))` for tiling arr n_times in the first new-axis dimension

- Ref.
    - https://stackoverflow.com/a/54269982

In [ ]:
def prepend_dim(t, n_times=1, in_place=False):
    """
    Prepend dimension (of size=1) to the input tensor `t`'s dimension
    n_times
    """
    if in_place:
        _t = torch.empty_like(t)
        _t.copy_(t)
    else: 
        _t = t
        
    for i in range(n_times):
        _t.unsqueeze_(dim=0)
    return _t

In [ ]:
import imagen as ig

## Generate filters

In [ ]:
gen_sine = ig.SineGrating()
gen_hp = ig.HalfPlane()

hp_np = gen_hp()
sg_np = gen_sine()

In [ ]:
hp_t = torch.from_numpy(hp_np).float()
sg_t = torch.from_numpy(sg_np).float()
zeros_t = torch.zeros_like(hp_t)
sg_3d = torch.stack([hp_t.t(),zeros_t, zeros_t])# hp_t, sg_t])
print(sg_3d.shape)

In [ ]:
sg_batch = prepend_dim(sg_3d, n_times=1)
sg_batch.shape

In [ ]:
# 2dimensional kernel for x directional gradient that will be applied to each clannel
kx = np.array(
    [[-1,0,1],[-1,0,1],[-1,0,1]],
    dtype=np.float32)
ky = kx.T

### Efficient way to create a 1-dim higher tile by repeating an array

In [ ]:
kx_3d = np.broadcast_to(kx, (3, *kx.shape))
ky_3d = np.broadcast_to(ky, (3, *ky.shape))

In [ ]:
k1 = torch.from_numpy(kx_3d)
k2 = torch.from_numpy(ky_3d)
k = torch.stack([k1,k2])
k.shape


Using `view` with `np.broadcast_to` this way is more memory, computation(no copying) efficient than, eg:
```python
# Wasteful 
grad_x = [[-1,0,1], [-1,0,1], [-1,0,1]]
grad_x = np.asarray(grad_x, dtype=np.float32)
grad_y = grad_x.T
rgb_grad_x = np.stack([grad_x, grad_x, grad_x])
rgb_grad_y = np.stack([grad_y, grad_y, grad_y])
rgb_filters = np.stack([rgb_grad_x, rgb_grad_y])

conv.weight.data = torch.from_numpy(rgb_filters.astype(np.float32) )
print(conv.weight.data.shape)

```

### Convolutional layer

In [ ]:
conv = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, padding=1)
# Note: conv.weight.shape = (out_channels, in_channels, kernel_size, kernel_size)

In [ ]:
conv.weight.data = k

In [ ]:
sg_gradx = conv(sg_batch)
sg_gradx.shape

In [ ]:
out = sg_gradx.detach().squeeze().numpy()

In [ ]:
%%opts Image (cmap='gray') [height=300, width=300, colorbar=True] {+axiswise}
hv.Image(hp_np.T)+ hv.Image(out[0,:,:], label='gradx') + hv.Image(out[1,:,:], label='grady')